<a href="https://colab.research.google.com/github/YanfuOu/CIS545/blob/Master/Yanfu_HW2_FA2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 5450 Homework 2: SQL and DuckDB
## Due: Friday, October 11 2024, 10:00pm EST
### Worth 95 points in total (25 manually graded)

Welcome to Homework 2! By now, you should be familiar with the world of data science and the Pandas library. This assignment focuses on helping you get to grips with two new tools: SQL and DuckDB.

Through this homework, we will be working with SQL by exploring a [Indego](https://www.rideindego.com/) dataset containing bike rides, stations and weather data. We will then expand our exploration with DuckDB and finish by comparing the two with Pandas.

 <!-- We will finish off the homework with some text analysis. -->

We are introducing a lot of new things in this homework, and this is often where students start to get lost. Thus, we **strongly** encourage you to review the slides/material as you work through this assignment.

**Before you begin:**
- Be sure to click "Copy to Drive" to make sure you're working on your own personal version of the homework
- Check the pinned FAQ post on Ed for updates! If you have been stuck, chances are other students have also faced similar problems.

##Part 0: Libraries and Set Up Jargon

In [1]:
import pandas as pd

In [2]:
!pip3 install penngrader-client
!pip install sqlalchemy==1.4.46
!pip install pandasql
!pip install geopy
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.46 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=73bd09ecc7b29a9630ab506cb7203252b14f99d148b868ef1c37da2f7cd94638
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.5 MB/s et

In [3]:
from penngrader.grader import *
import pandas as pd
import datetime as dt
import geopy.distance as gp
import matplotlib.image as mpimg
import plotly.express as px
import re
import pandasql as ps #SQL on Pandas Dataframe
import nltk
nltk.download('punkt')
import duckdb
from wordcloud import WordCloud
from matplotlib.dates import date2num
import matplotlib.pyplot as plt
from PIL import Image
# from collections import Counter
# import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Three datasets we're using
! wget -nc https://storage.googleapis.com/penn-cis5450/indego_trips.csv
! wget -nc https://storage.googleapis.com/penn-cis5450/indego_stations.csv
! wget -nc https://storage.googleapis.com/penn-cis5450/weather_2022_PHL.csv

--2024-12-07 18:57:43--  https://storage.googleapis.com/penn-cis5450/indego_trips.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.96.207, 108.177.119.207, 108.177.127.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.96.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2033339 (1.9M) [text/csv]
Saving to: ‘indego_trips.csv’

indego_trips.csv    100%[===================>]   1.94M  3.10MB/s    in 0.6s    

2024-12-07 18:57:44 (3.10 MB/s) - ‘indego_trips.csv’ saved [2033339/2033339]

--2024-12-07 18:57:44--  https://storage.googleapis.com/penn-cis5450/indego_stations.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.96.207, 108.177.119.207, 108.177.127.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.96.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9052 (8.8K) [text/csv]
Saving to: ‘indego_stations.csv’

indego_stations.csv 

###PennGrader Setup

In [5]:
# PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW
# TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 61033141 # YOUR PENN-ID GOES HERE AS AN INTEGER #
SECRET = STUDENT_ID

In [6]:
%%writefile config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing config.yaml


In [7]:
grader = PennGrader('config.yaml', 'cis5450_fall24_HW2', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 61033141

Make sure this correct or we will not be able to store your grade


## Biking in Philadelphia


I'm sure in your time in Philadelphia so far you've come across these blue bikes and stations. Indego is the company responsible for this bike sharing ride system, and they make data on bike trips available to the public. This data can not only be useful to get information of how people in Philly use bikes, but it can give information on the most visited places in the city which can be useful for city planners and business owners.


In this homework, we'll be exploring some data about bikes including:

*   Trips: data about bike trips during the first week of October 2022.

*   Stations: data about bike stations, their ID and Name.

*   Weather: data about the weather in Philadelphia during 2022.


We'll be parsing this data into dataframes and relations, and then exploring how to query and assemble the tables into results. We will primarily be using DuckDB, but for some of the initial questions, we will ask you to perform the same operations in Pandas as well, so as to familiarize you with the differences and similarities of the two.


## Part 1: Load & Process our Datasets [15 points total]

Before we get into the data, we first need to load and clean our datasets.

### Metadata
You'll be working with three CSV files:
- `indego_trips.csv`
- `indego_stations.csv`
- `weather_2022_PHL.csv`

The file `indego_trips.csv` contains data about each trip, like the origin station, destination station and duration.

The file `indego_stations.csv` includes information about stations and their status in January 2023.

The file `weather_2022_PHL.csv` has one row per day during 2022 and shows weather information.


**TODO**:
* Load `indego_trips.csv` and save the data to a dataframe called `trips_df`.
* Load `indego_stations.csv` and save the data to a dataframe called `stations_df`.
* Load `weather_2022_PHL.csv` and save the data to a dataframe called `weather_df`.


In [8]:
# TODO: Import the datasets to pandas dataframes -- make sure the dataframes are named correctly!
trips_df = pd.read_csv("indego_trips.csv")
stations_df = pd.read_csv("indego_stations.csv")
weather_df = pd.read_csv("weather_2022_PHL.csv")


In [9]:
# view trips_df using .head() to make sure the import was successful
trips_df.head()
trips_test_df = trips_df.head(20)



In [10]:
print(trips_test_df)

      trip_id  duration      start_time        end_time  start_station  \
0   536547052         9  10/1/2022 0:00  10/1/2022 0:09         3190.0   
1   536547050        19  10/1/2022 0:01  10/1/2022 0:20         3168.0   
2   536547048        12  10/1/2022 0:02  10/1/2022 0:14         3190.0   
3   536547046         5  10/1/2022 0:03  10/1/2022 0:08         3075.0   
4   536547044        11  10/1/2022 0:03  10/1/2022 0:14         3050.0   
5   536547042         4  10/1/2022 0:03  10/1/2022 0:07         3248.0   
6   536547040        30  10/1/2022 0:08  10/1/2022 0:38         3041.0   
7   536547038        18  10/1/2022 0:08  10/1/2022 0:26         3041.0   
8   536547036        11  10/1/2022 0:08  10/1/2022 0:19         3073.0   
9   536547034         4  10/1/2022 0:12  10/1/2022 0:16         3005.0   
10  536547032        15  10/1/2022 0:16  10/1/2022 0:30         3010.0   
11  536547030        11  10/1/2022 0:16  10/1/2022 0:27         3010.0   
12  536547028        86  10/1/2022 0:1

In [11]:
# view stations_df using .head() to make sure the import was successful

stations_df.head()


,station_id,station_name,day_of_go_live_date,status,zone
0,3000,Virtual Station,4/23/2015,Active,0
1,3004,Municipal Services Building Plaza,4/23/2015,Active,0
2,3005,"Welcome Park, NPS",4/23/2015,Active,0
3,3006,40th & Spruce,4/23/2015,Active,2
4,3007,"11th & Pine, Kahn Park",4/23/2015,Active,0


In [12]:
# view weather_df using .head() to make sure the import was successful

weather_df.head()

,date,actual_mean_temp,actual_min_temp,actual_max_temp,average_min_temp,average_max_temp,record_min_temp,record_max_temp,record_min_temp_year,record_max_temp_year,actual_precipitation,average_precipitation,record_precipitation
0,1/1/2022,35,28,42,27,41,4,64,1881,2005,0.00,0.09,1.60
1,1/2/2022,37,29,44,26,41,7,67,1968,1876,0.00,0.10,1.68
2,1/3/2022,36,28,44,26,41,-3,63,1879,2000,0.72,0.09,1.50
3,1/4/2022,50,42,58,26,41,2,68,1918,1950,0.37,0.10,1.51
4,1/5/2022,37,24,50,26,40,-2,66,1904,1997,0.00,0.10,2.24


###1.1 Data Preprocessing

Next, we are going to want to clean up our dataframes, namely `trips_df` and `stations_df`, by 1) fixing columns, 2) changing datatypes, 3) handling nulls.

First, let us view the first few rows of `trips_df`. You may also call `.info()` and additionally check the cardinality of each column to view the specifics of the dataframe. This is a good first step to take for Exploratory Data Analysis (EDA).

#### 1.1.1 Cleaning `trips_df` [8 points]

`.info()` gives us meaningful information regarding columns, their types, and the amount of nulls, based on which we can now clean our dataframe.

Perform these steps and save results on a new dataframe: `trips_cleaned_df`

**TODO**:
* Drop the column `plan_duration`. We already have that information in the column `passholder_type`, which is more understandable.
* Drop the rows where `end_station` is 3000. This is a virtual station used for maintainance, and doesn't represent a real trip.
* Drop all rows with null values.
* Cast the columns:
  -  `start_time`, `end_time`, `trip_route_category`, `passholder_type`, `bike_type` as **string**. (Cast to 'string' and not 'str')
  - `bike_id` as **int**.
* Sort results by `trip_id` ascending
* Reset and drop the index and save results as `trips_cleaned_df`

After performing these steps, `trips_cleaned_df` should have the following schema:

**Final Schema**:
>trip_id | duration | start_time | end_time | start_station | start_lat | start_lon | end_station |  end_lat | end_lon |  bike_id | trip_route_category | passholder_type | bike_type
>--- | --- | --- |--- | --- | --- | --- |--- | --- | --- |--- |--- | --- | --- |

In [13]:
#view info of trips_df
trips_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15502 entries, 0 to 15501
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   trip_id              15502 non-null  int64  
 1   duration             15502 non-null  int64  
 2   start_time           15502 non-null  object 
 3   end_time             15502 non-null  object 
 4   start_station        15477 non-null  float64
 5   start_lat            15477 non-null  float64
 6   start_lon            15477 non-null  float64
 7   end_station          15477 non-null  float64
 8   end_lat              15255 non-null  float64
 9   end_lon              15255 non-null  float64
 10  bike_id              15477 non-null  object 
 11  plan_duration        15477 non-null  float64
 12  trip_route_category  15477 non-null  object 
 13  passholder_type      15477 non-null  object 
 14  bike_type            15477 non-null  object 
dtypes: float64(7), int64(2), object(6)
m

In [14]:
# TODO: drop plan_duration
trips_df = trips_df.drop(columns="plan_duration")

In [15]:
# TODO: drop rows with irrelevant end_station
trips_df[trips_df['end_station']!= 3000]

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,trip_route_category,passholder_type,bike_type
0,536547052,9,10/1/2022 0:00,10/1/2022 0:09,3190.0,39.948921,-75.169907,3158.0,39.925522,-75.169037,23553,One Way,Indego30,electric
1,536547050,19,10/1/2022 0:01,10/1/2022 0:20,3168.0,39.951340,-75.173943,3034.0,39.933151,-75.162483,22190,One Way,Indego30,electric
2,536547048,12,10/1/2022 0:02,10/1/2022 0:14,3190.0,39.948921,-75.169907,3158.0,39.925522,-75.169037,18789,One Way,Indego30,electric
3,536547046,5,10/1/2022 0:03,10/1/2022 0:08,3075.0,39.967178,-75.161247,3039.0,39.971569,-75.159927,23383,One Way,Indego30,electric
4,536547044,11,10/1/2022 0:03,10/1/2022 0:14,3050.0,39.953388,-75.154259,3075.0,39.967178,-75.161247,5367,One Way,Indego30,standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15497,540422628,10,10/7/2022 23:47,10/7/2022 23:57,3161.0,39.954861,-75.180908,3212.0,39.963829,-75.181770,11044,One Way,Indego30,standard
15498,540422627,6,10/7/2022 23:49,10/7/2022 23:55,3010.0,39.947109,-75.166183,3150.0,39.928841,-75.170212,22989,One Way,Indego30,electric
15499,540422626,6,10/7/2022 23:51,10/7/2022 23:57,3272.0,39.964050,-75.220482,3119.0,39.966740,-75.207993,17081,One Way,Indego30,electric
15500,540422625,8,10/7/2022 23:51,10/7/2022 23:59,3077.0,39.972069,-75.163513,3088.0,39.969841,-75.141800,23676,One Way,Indego30,electric


In [16]:
# TODO: drop rows with null values
trips_df = trips_df.dropna()

In [17]:
# TODO: cast the types of the columns
#start_time, end_time, trip_route_category, passholder_type, bike_type as string. (Cast to 'string' and not 'str')
#bike_id as int.
trips_df = trips_df.astype({
    'start_time' : 'string',
    'end_time' : 'string',
    'trip_route_category' : 'string',
    'passholder_type' : 'string',
    'bike_type' : 'string',
    'bike_id' : int

})

In [18]:
# TODO: sort the results
trips_df = trips_df.sort_values(by="trip_id")

In [19]:
# TODO: drop the index and save the results
trips_cleaned_df = trips_df.reset_index(drop=True)

In [20]:
# 4 points
grader.grade(test_case_id = 'test_cleaning_trips', answer = trips_cleaned_df)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


Now we are going to clean up the start_time and end_time columns so that they are easier to use. We will be using Regex in this section to separate out the date and the time from the entries.

**TODO**:
* Fill in the Regex patterns to retrieve first the date, and then the time, found in each entry.
* Extract the relevant parts of each column and populate new columns called the following: `date_start`, `time_start`, `date_end`, `time_end`. Note that the datetime type does contain both date and time, but that we are wanting to explore your Regex capabilities :)
* Cast the new date columns as `datetime64[ns]`, using pd.to_datetime() and the format as '%m/%d/%Y'
* Remove columns `start_time`, `end_time`

In [21]:
#view trips_cleaned_df using .head()
trips_cleaned_df.head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,trip_route_category,passholder_type,bike_type
0,536546924,3,10/1/2022 1:50,10/1/2022 1:53,3188.0,39.904709,-75.173401,3188.0,39.904709,-75.173401,11708,Round Trip,Indego30,standard
1,536546925,3,10/1/2022 1:50,10/1/2022 1:53,3188.0,39.904709,-75.173401,3188.0,39.904709,-75.173401,3483,Round Trip,Indego30,standard
2,536546927,7,10/1/2022 1:48,10/1/2022 1:55,3010.0,39.947109,-75.166183,3064.0,39.938400,-75.173271,14655,One Way,Indego30,standard
3,536546929,7,10/1/2022 1:48,10/1/2022 1:55,3010.0,39.947109,-75.166183,3064.0,39.938400,-75.173271,12987,One Way,Indego30,standard
4,536546932,6,10/1/2022 1:41,10/1/2022 1:47,3004.0,39.953781,-75.163742,3206.0,39.951740,-75.170631,22554,One Way,Day Pass,electric


In [22]:
# TODO: cast the types of start_time and end_time to 'string'
trips_cleaned_df = trips_cleaned_df.astype({
    'start_time' : 'string',
    'end_time' :  'string',
})
trips_cleaned_df.dtypes

,0
trip_id,int64
duration,int64
start_time,string[python]
end_time,string[python]
start_station,float64
start_lat,float64
start_lon,float64
end_station,float64
end_lat,float64
end_lon,float64


In [23]:
trips_cleaned_df

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,trip_route_category,passholder_type,bike_type
0,536546924,3,10/1/2022 1:50,10/1/2022 1:53,3188.0,39.904709,-75.173401,3188.0,39.904709,-75.173401,11708,Round Trip,Indego30,standard
1,536546925,3,10/1/2022 1:50,10/1/2022 1:53,3188.0,39.904709,-75.173401,3188.0,39.904709,-75.173401,3483,Round Trip,Indego30,standard
2,536546927,7,10/1/2022 1:48,10/1/2022 1:55,3010.0,39.947109,-75.166183,3064.0,39.938400,-75.173271,14655,One Way,Indego30,standard
3,536546929,7,10/1/2022 1:48,10/1/2022 1:55,3010.0,39.947109,-75.166183,3064.0,39.938400,-75.173271,12987,One Way,Indego30,standard
4,536546932,6,10/1/2022 1:41,10/1/2022 1:47,3004.0,39.953781,-75.163742,3206.0,39.951740,-75.170631,22554,One Way,Day Pass,electric
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15250,540422839,100,10/7/2022 21:36,10/7/2022 23:16,3045.0,39.947922,-75.162369,3045.0,39.947922,-75.162369,22404,Round Trip,Day Pass,electric
15251,540422840,102,10/7/2022 21:35,10/7/2022 23:17,3045.0,39.947922,-75.162369,3045.0,39.947922,-75.162369,23321,Round Trip,Day Pass,electric
15252,540422841,102,10/7/2022 21:34,10/7/2022 23:16,3045.0,39.947922,-75.162369,3045.0,39.947922,-75.162369,22950,Round Trip,Day Pass,electric
15253,540422860,50,10/7/2022 21:27,10/7/2022 22:17,3167.0,39.950249,-75.160278,3039.0,39.971569,-75.159927,14533,One Way,Indego30,standard


In [24]:
# TODO: fill in the pattern to retrieve the date of each entry
# HINT: think about the unique syntax of the date section, the different variations it can appear in
date_pattern = r"^[0-9]{1,2}/[0-9]{1,2}/[0-9]{1,2}"

In [25]:
# TODO: fill in the pattern to retrieve the time of each entry
# HINT: think about the unique syntax of the time section, the different variations it can appear in
time_pattern = r"[0-9]{1,2}:[0-9]{1,2}$"

In [26]:
# TODO: populate columns date_start, time_start, date_end, time_end
trips_cleaned_df['date_start'] = trips_cleaned_df['start_time'].apply(lambda x: re.findall(date_pattern, x)[0])
trips_cleaned_df['time_start'] = trips_cleaned_df['start_time'].apply(lambda x: re.findall(time_pattern, x)[0])
trips_cleaned_df['date_end'] = trips_cleaned_df['end_time'].apply(lambda x: re.findall(date_pattern, x)[0])
trips_cleaned_df['time_end'] = trips_cleaned_df['end_time'].apply(lambda x: re.findall(time_pattern, x)[0])

In [27]:
# TODO: cast the date columns to as datetime64[ns]
trips_cleaned_df['date_start'] = pd.to_datetime(trips_cleaned_df['date_start'], format='%m/%d/%y')
trips_cleaned_df['date_end'] = pd.to_datetime(trips_cleaned_df['date_end'], format='%m/%d/%y')

In [28]:
# TODO: drop the start_time and end_time
trips_cleaned_df = trips_cleaned_df.drop(columns=['start_time', 'end_time'])

In [29]:
# 4 points
grader.grade(test_case_id = 'test_regex_trips', answer = trips_cleaned_df)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.1.2 Processing Stations [3 Points]

`stations_df` contains information on Indego stations across the city. We will clean this df by removing Inactive stations and stations created after October 2022.

Perform these steps and assign the cleaned dataframe to `stations_cleaned_df`.

**TODO**:
* **Drop** the stations that have an **Inactive** status.
* **Cast** column `day_of_go_live_date` as datetime64[ns].
* **Drop** the stations that were created **after 10/7/2022** since this is the last date of rides we are analyzing.
* **Drop** the columns `day_of_go_live_date` and `status`
* **Create** a new column called `is_west_philly` that is True if zone is 2 or 3 and False otherwise.
* **Save** the resulting dataframe as `stations_cleaned_df`, and sort it by `station_id` ascending


After performing these steps, `stations_cleaned_df` should have the following schema:


**Final Schema**:
>station_id | station_name | zone | is_west_philly
>--- | --- | --- |--- |

In [30]:
#view info of stations_df
stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   station_id           203 non-null    int64 
 1   station_name         203 non-null    object
 2   day_of_go_live_date  203 non-null    object
 3   status               203 non-null    object
 4   zone                 203 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 8.1+ KB


In [31]:
# TODO: Drop the stations that have an Inactive status.
stations_df = stations_df[stations_df['status'] != 'Inactive']

In [32]:
# TODO: Cast column day_of_go_live_date as datetime64[ns].
#stations_df['day_of_go_live_date'] = pd.to_datetime(stations_df['day_of_go_live_date'], format="%m/%d/%y")
stations_df["day_of_go_live_date"] = pd.to_datetime(stations_df["day_of_go_live_date"], format= "%m/%d/%Y")

<ipython-input-32-ee7285ecc423>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_df["day_of_go_live_date"] = pd.to_datetime(stations_df["day_of_go_live_date"], format= "%m/%d/%Y")


In [33]:
# TODO: Drop the stations that were created after 10/7/2022.
cutoff = pd.to_datetime('10/7/2022', format ="%m/%d/%Y")
stations_df = stations_df[stations_df['day_of_go_live_date'] <= cutoff]

In [34]:
# TODO: Drop day_of_go_live_date and status columns
stations_df = stations_df.drop(columns=['day_of_go_live_date', 'status'])

In [35]:
# TODO: Create a new column called is_west_philly that is True if zone is 2 or 3 and False otherwise.
#stations_df['is_west_philly'] = (stations_df['zone'] == 2) | (stations_df['zone'] == 3)
stations_df['is_west_philly'] = stations_df['zone'].apply(lambda x: True if (x == 2 or x == 3) else False)

In [36]:
# TODO: Sort by station_id ascending
stations_df = stations_df.sort_values(by='station_id', ascending=True)

In [37]:
# TODO: Reset and drop the index, and save the resulting dataframe as stations_cleaned_df
stations_cleaned_df = stations_df.reset_index(drop=True)

In [38]:
# 3 points
grader.grade(test_case_id = 'test_cleaning_stations', answer = stations_cleaned_df)

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.1.3 Cleaning the weather [4 Points]
Then, let's clean `weather_df` and make it usable. We are going to make two different datasets, one for the actual data, and another for the record-holding data.

**TODO**:
* Create `actual_weather_cleaned_df` and only keep the following 5 columns:
     * `date`, `actual_mean_temp`, `actual_min_temp`, `actual_max_temp`, `actual_precipitation`
* Create `record_weather_cleaned_df` and only keep the following 4 columns:
     * `date`, `record_min_temp`, `record_max_temp`, `record_precipitation`

Then for both datasets:
* Convert column `date` into type `datetime64[ns]`.
* Keep only the rows from 9/1/2022 to 10/31/2022, inclusive.
* Sort by column `date` descending.
* Reset and drop the index.

After performing these steps, `actual_weather_cleaned_df` should have the following schema:

**Final Schema**:
>date | actual_mean_temp | actual_min_temp | actual_max_temp | actual_precipitation
>--- | --- | --- |--- |--- |

... and `record_weather_cleaned_df` should have the following schema:

**Final Schema**:
>date | record_min_temp | record_max_temp | record_precipitation
>--- | --- | --- |--- |

In [39]:
#view info of weather_df
weather_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   365 non-null    object 
 1   actual_mean_temp       365 non-null    int64  
 2   actual_min_temp        365 non-null    int64  
 3   actual_max_temp        365 non-null    int64  
 4   average_min_temp       365 non-null    int64  
 5   average_max_temp       365 non-null    int64  
 6   record_min_temp        365 non-null    int64  
 7   record_max_temp        365 non-null    int64  
 8   record_min_temp_year   365 non-null    int64  
 9   record_max_temp_year   365 non-null    int64  
 10  actual_precipitation   365 non-null    float64
 11  average_precipitation  365 non-null    float64
 12  record_precipitation   365 non-null    float64
dtypes: float64(3), int64(9), object(1)
memory usage: 37.2+ KB


In [40]:
# TODO: create actual_weather_cleaned_df
actual_weather_cleaned_df = weather_df[['date', 'actual_mean_temp', 'actual_min_temp', 'actual_max_temp', 'actual_precipitation']]

In [41]:
# TODO: create record_weather_cleaned_df
record_weather_cleaned_df = weather_df[['date', 'record_min_temp', 'record_max_temp', 'record_precipitation']]

In [42]:
# TODO: for both datasets, convert column 'date' into type datetime64[ns]
actual_weather_cleaned_df['date'] = pd.to_datetime(actual_weather_cleaned_df['date'], format = '%m/%d/%Y')
record_weather_cleaned_df['date'] = pd.to_datetime(record_weather_cleaned_df['date'], format = '%m/%d/%Y')

<ipython-input-42-2228c1bc2d64>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_weather_cleaned_df['date'] = pd.to_datetime(actual_weather_cleaned_df['date'], format = '%m/%d/%Y')
<ipython-input-42-2228c1bc2d64>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  record_weather_cleaned_df['date'] = pd.to_datetime(record_weather_cleaned_df['date'], format = '%m/%d/%Y')


In [43]:
# TODO: for both datasets, keep only the rows from 9/1/2022 to 10/31/2022, inclusive
cutOffL = pd.to_datetime("9/1/2022", format="%m/%d/%Y")
cutOffH = pd.to_datetime("10/31/2022", format="%m/%d/%Y")
actual_weather_cleaned_df = actual_weather_cleaned_df[(actual_weather_cleaned_df['date'] >= cutOffL) & (actual_weather_cleaned_df['date'] <= cutOffH)]
record_weather_cleaned_df = record_weather_cleaned_df[(record_weather_cleaned_df['date'] >= cutOffL) & (record_weather_cleaned_df['date'] <= cutOffH)]

In [44]:
# TODO: for both datasets, sort by column 'date' descending
actual_weather_cleaned_df = actual_weather_cleaned_df.sort_values(by='date', ascending=False)
record_weather_cleaned_df = record_weather_cleaned_df.sort_values(by='date', ascending=False)

In [45]:
# TODO: for both datasets, reset and drop the index
actual_weather_cleaned_df = actual_weather_cleaned_df.reset_index(drop=True)
record_weather_cleaned_df = record_weather_cleaned_df.reset_index(drop=True)

In [46]:
# 4 points
grader.grade(test_case_id = 'test_cleaning_weather', answer = [actual_weather_cleaned_df, record_weather_cleaned_df])

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Part 2: DuckDB [55 points total]

### 👇👇👇 IMPORTANT: Pay VERY CLOSE attention to this style guide! 👇👇👇

The typical flow to use duckdb is as follows:
1. Write a SQL query in the form of a string
    - **String Syntax:** use triple quotes `"""<your query>"""` to write multi-line strings
    - **Aliases are your friend:** if there are very long table names or you find yourself needed to declare the source (common during join tasks), it's almost always optimal to alias your tables with short INTUITIVE alias names
    - **New Clauses New Line:** each of the main SQL clauses (`SELECT`, `FROM`, `WHERE`, etc.) should begin on a new line
    - **Use Indentation:** if there are many components for a single clause, separate them out with new <ins>indented</ins> lines.

    Example below:
    ```SQL
    """
    SELECT ltn.some_id, SUM(stn.some_value) AS total
    FROM long_table_name AS ltn
         INNER JOIN short_table_name AS stn
            ON ltn.common_key = stn.common_key
         INNER JOIN med_table_name AS mtn
            ON ltn.other_key = mtn.other_key
    WHERE ltn.col1 > value
         AND stn.col2 <= another_value
         AND mtn.col3 != something_else
    GROUP BY ltn.some_id
    ORDER BY total
    """
    ```
2. Run the query using **duckdb.sql(your_query)**

Duckdb and pandasql are convenient in that they allow you to reference the dataframes that are currently defined in your notebook, so you will be able to fully utilize the dataframes that you have created above!

Given that it is a brand new language, we wanted to give you a chance to directly compare the similarities/differences of the pandas that you already know and the SQL that you are about to learn. SQL queries may take up to a minute to run.

**⚠WARNING: DO NOT USE PANDAS FOR SQL QUESTIONS OR VICE-VERSA! OTHERWISE, YOU WON'T RECEIVE CREDITS FOR THESE QUESTIONS**

In [47]:
#TODO: Run this cell to understand how Duck DB connects to local dataframes and queries them
test_duckdb_query = """
SELECT *
FROM actual_weather_cleaned_df
LIMIT 10
"""

duckdb.sql(test_duckdb_query)

┌─────────────────────┬──────────────────┬─────────────────┬─────────────────┬──────────────────────┐
│        date         │ actual_mean_temp │ actual_min_temp │ actual_max_temp │ actual_precipitation │
│    timestamp_ns     │      int64       │      int64      │      int64      │        double        │
├─────────────────────┼──────────────────┼─────────────────┼─────────────────┼──────────────────────┤
│ 2022-10-31 00:00:00 │               47 │              40 │              54 │                  0.0 │
│ 2022-10-30 00:00:00 │               53 │              45 │              60 │                  0.0 │
│ 2022-10-29 00:00:00 │               60 │              50 │              70 │                 0.03 │
│ 2022-10-28 00:00:00 │               63 │              49 │              76 │                  0.0 │
│ 2022-10-27 00:00:00 │               57 │              46 │              67 │                  0.0 │
│ 2022-10-26 00:00:00 │               59 │              50 │              67 │    

### 2.1 Comparing Rides by Passholders [17 points]

#### 2.1.1 How many rides were taken per passholder type? [3 points]

The dataframe `trips_cleaned_df` contains information for each ride. We want to know which of these rides were taken per passholder type (Day Pass, Indego30, Indego365), and how many such rides were taken.

**TODO:**
*  Create a DuckDB query that counts the number of trips taken per passholder type.
*  The resulting table should have the column `passholder_type` that contains the pass names and another column `trip_count` that is the count of the number of trips taken on that pass.
*  Sort this table by `trip_count` in descending order.

**Note:** Do not modify the lines where we are calling the Duck DB SQL engine, as this line converts your results to a Pandas dataframe for our test cases.

**Hint:** It would be helpful to use the `GROUP BY` and `ORDER BY` statements in SQL. Documentation for the `GROUP BY` statement can be found [here](https://www.w3schools.com/sql/sql_groupby.asp) and `ORDER BY` statement can be found [here](https://www.w3schools.com/sql/sql_orderby.asp).

**Final Schema:**

>passholder_type | trip_count
>--- | ---

In [48]:
# TODO: Use Duck DB to get the total number of trips by passholder type
passholder_count_query = """
SELECT passholder_type, COUNT(trip_id) AS trip_count FROM trips_cleaned_df
GROUP BY passholder_type
ORDER BY COUNT(trip_id) DESC
"""

passholder_count = duckdb.sql(passholder_count_query).df()

In [49]:
# 3 Points
grader.grade(test_case_id = 'test_passholder_count', answer = (passholder_count, passholder_count_query))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 2.1.2 What was the average distance between the start and end point of the ride by passholder type? [5 points]

We are now interested in what the average displacement was between the start and end of the ride by each passholder type.

**TODO:**
*  Create a (Python) function called `calculate_distance` that returns the distance in miles between two longitude/latitude coordinates. The input parameters should be `start_lat`, `start_lon`, `end_lat`, `end_lon`
*  Create a function in Duck DB using the `create_function` functionality in Duck DB. Name this function `distance`. The input/output datatypes for this function should all be `float`.
*  Use this function **inside** a Duck DB query to find the average trip displacement by passholder type for **non-roundtrip rides** from `trips_cleaned_df`.
*  Sort the table by the average displacement in descending order.

Documentation on using Python functions in Duck DB can be found [here](https://duckdb.org/docs/api/python/function). The "Python Function API" and "Creating Functions" sections would be most helpful.

**Hint:** Use the GeoPy Distance module to help create your function, which has already been imported for you at the top of this notebook! Documentation on this module can be found [here](https://geopy.readthedocs.io/en/stable/#module-geopy.distance).

**Final Schema:**
>passholder_type | avg_displacement
>--- | ---

In [50]:
# TODO: Finish the rest of the calculate_distance function by using the GeoPy Distance module.
from geopy import distance
def calculate_distance(start_lat, start_lon, end_lat, end_lon):
  pt1 = (start_lat, start_lon)
  pt2 = (end_lat, end_lon)
  return distance.distance(pt1, pt2).miles

The following test case is not graded but given to you all as a spot check. If you don't receive a hint from the test case you are good to go!

In [51]:
# 0 point
grader.grade(test_case_id = 'test_distance_function', answer = calculate_distance(30.4515, -91.1871, 29.9509, -90.0758))

Correct! You earned 0/0 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [52]:
# DO NOT EDIT
try:
  duckdb.remove_function("distance")
except:
  pass

# TODO: Use the calculate_distance function to create a Duck DB function with the correct parameters
duckdb.create_function("distance",calculate_distance, [float, float, float, float], float )

In [53]:
# TODO: Use Duck DB to find the average displacement for non-roundtrip rides for each passholder type.
distance_query = """
SELECT passholder_type, AVG(distance(start_lat, start_lon, end_lat, end_lon)) AS avg_displacement
FROM trips_cleaned_df
WHERE trip_route_category != 'Round Trip'
GROUP BY passholder_type
ORDER BY avg_displacement DESC
"""
passholder_distance = duckdb.sql(distance_query).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [54]:
# 5 points
grader.grade(test_case_id = 'test_passholder_distance', answer = passholder_distance)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 2.1.3 What is the distribution of electric vs standard bikes for each type of pass? [4 points]

**TODO:**
*  Create a DuckDB query that finds the percentage of rides that used an electric bike for each type of pass from `trips_cleaned_df`.
*  The resulting table should have the column `passholder_type` and `electric_pct`, which shows what percentage of the rides taken for that pass were on electric bikes.
*  For the `electric_pct` column, scale the percentages to be between 0 and 100 (e.g. 40.4% = 40.4) and rounded to 1 decimal place.
*  Sort the table by `electric_pct` in descending order.
*  Do not modify the variable name of the query string (passholder_bike_type_query), as this variable will be passed into the test case.

**Hint:** For documentation on rounding in SQL, see the documentation [here](https://www.w3schools.com/sql/func_mysql_round.asp). For documentation on the `CASE` statement, see the documentation [here](https://www.w3schools.com/sql/sql_case.asp).

**Final Schema:**
>passholder_type | electric_pct
>--- | ---

In [55]:
# TODO: Create a Duck DB query that finds the percentage of rides on an electric bike for each type of passholder.
passholder_bike_type_query = """
SELECT passholder_type,
ROUND(
100 * SUM(CASE WHEN bike_type == 'electric' THEN 1 ELSE 0 END) / COUNT(bike_type),
1)
AS electric_pct
FROM trips_cleaned_df
GROUP BY passholder_type
ORDER BY electric_pct DESC

"""
passholder_bike_type = duckdb.sql(passholder_bike_type_query).df()
passholder_bike_type

,passholder_type,electric_pct
0,Day Pass,57.1
1,Indego30,49.4
2,Indego365,41.4


In [56]:
# 4 points
grader.grade(test_case_id = 'test_passholder_bike_type', answer = (passholder_bike_type, passholder_bike_type_query))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 2.1.4 What are the most popular rides for each day of the week? [4 points]

We want to understand what are the most popular days of the week for bike rides for each type of pass in `trips_cleaned_df`.

**TODO:**
*  Create a query using Duck DB that returns a table with the following schema:

>Day of Week | Number of Days | Day Pass | Indego30 | Indego365
>--- | --- | --- |--- | ---

*  The `Day of Week` column indicates the specific day of the week when the rides occurred. The `Day Pass`, `Indego30`, and `Indego365` columns represent the total number of rides for each type of pass on that particular day of the week. For example, if a row lists "Monday" in the Day of Week column, the value in the Day Pass column should reflect the total number of rides taken with a Day Pass on all Mondays throughout the dataset.
*  `Number of Days` is the number of dates in our dataset that fall on that row's day of the week.
*  Sort the dataframe to start from Monday and end on Sunday.

**Hint:** Look at the documentation [here](https://www.w3schools.com/sql/func_mysql_dayname.asp) to extract day of week names in SQL. Look at the documentation [here](https://www.w3schools.com/sql/sql_distinct.asp) to figure out how to count the distinct dates for each day of the week in the `Number of Days` column.

**Hint:** If we are defining a column alias in SQL as a string with spaces (e.g. "Day of Week"), we have to put it in double quotes inside of the query. To reference a string, only single quotes are needed (e.g. 'Monday').

**Hint:** To sort the dataframe by the correct days of the week, consider using a [CASE statement](https://www.w3schools.com/sql/sql_case.asp) to assign each day of the week to a number, then sorting by this assignment.

In [95]:
# TODO: Create a Duck DB query in the schema specified above that counts the number of bike rides per passholder type on every day of the week.
day_of_week_query = """
SELECT
  DAYNAME(date_start) AS "Day of Week",
  COUNT(DISTINCT date_start) AS "Number of Days",
  COUNT(CASE WHEN passholder_type = 'Day Pass' THEN 1 END) AS "Day Pass",
  COUNT(CASE WHEN passholder_type = 'Indego30' THEN 1 END) AS "Indego30",
  COUNT(CASE WHEN passholder_type = 'Indego365' THEN 1 END) AS "Indego365"

FROM trips_cleaned_df
GROUP BY "Day of Week"
ORDER BY
    CASE "Day of Week"
        WHEN 'Monday' THEN 1
        WHEN 'Tuesday' THEN 2
        WHEN 'Wednesday' THEN 3
        WHEN 'Thursday' THEN 4
        WHEN 'Friday' THEN 5
        WHEN 'Saturday' THEN 6
        WHEN 'Sunday' THEN 7
    END
"""
day_of_week = duckdb.sql(day_of_week_query).df()
day_of_week

,Day of Week,Number of Days,Day Pass,Indego30,Indego365
0,Monday,1,24,1935,734
1,Tuesday,1,166,2637,873
2,Wednesday,1,168,2579,797
3,Thursday,1,84,1416,383
4,Friday,1,12,463,109
5,Saturday,1,18,870,264
6,Sunday,1,13,1237,473


In [58]:
# 4 points
grader.grade(test_case_id = 'test_day_of_week_count', answer = day_of_week)

You earned 1/4 points.

Check that your counts for the 'Day Pass' are correct. The expected number of rides are [18, 13, 24, 166, 168, 84, 12] Check that your counts for the 'Day Pass' are correct. The expected number of rides are [870, 1237, 1935, 2637, 2579, 1416, 463] Check that your counts for the 'Day Pass' are correct. The expected number of rides are [264, 473, 734, 873, 797, 383, 109] Don't worry, you can re-submit and we will keep only your latest score.


#### Reflect: What do you think? [1 point]

Which day of the week did Indego30 passholders ride the most? What can you say about the distribution of the number of rides for each day of the week? Do we have enough data to draw a fair conclusion about our data or rider behavior?

In [59]:
# TODO: Answer the above reflection questions.
your_answer = "Indego30 riders rode the most on Thursday. For Indego30 passholders, weekdays appear to be the most popular compared to weekends. We can deduce that the users primarily utilize bikes for commuting to work during the week instead of using it for leisure activities]"

In [60]:
# 1 point
grader.grade(test_case_id = 'test_reflection', answer = your_answer)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.2 Destinations from Amtrak Station [9 points]
As you might imagine, Amtrak Station is a very important location for Indego, as there is a lot of people that either want to get there for a train ride or are exiting from there to visit Philadelphia. Thus, we want to learn about where people tend to go once they leave Amtrak Station.

You will create this dataset using SQL (Duck DB)

#### 2.2.1 Names not numbers [3 points]
We will now use `stations_cleaned_df` to modify our `trips_cleaned_df`. If you look at the rides data, you will notice that both start and end station columns have numbers corresponding to the station ID listed. Using the station ID listed in stations_df, we want to include station names instead of station IDs in our `trips_final_df`.

**TODO** (`SQL`):
* Create a function called `station_ids_to_names_dict` that uses `stations_cleaned_df` to map station IDs with their corresponding name. If a mapping does not exist from a station ID, return "Unknown" as the station name.
* Convert this Python function into a Duck DB function you can use in a query. Name this function `station_ids_dict`. The input type should be an int and the output type should be a string.
* Create a query with the final output being a new dataframe called `trips_final_df` which you will use in 2.2.2.
* Replace the IDs in the `start_station` and `end_station` columns with the corresponding station name.
* Sort the resulting table by the trip_id (in ascending order).

**Hint:** Case the stations IDs into integers before you feed them into your Duck DB function.

**Hint:** In order to convert the station IDs to station names, a Python dictionary may be helpful. See the documentation [here](https://www.w3schools.com/PYTHON/python_dictionaries.asp).

**Final Schema:**

> | trip_id | duration | start_station | start_lat | start_lon | end_station | end_lat | end_lon | bike_id | trip_route_category | passholder_type | bike_type | date_start | time_start | date_end | time_end |
> |---------|----------|---------------|-----------|-----------|-------------|---------|---------|---------|---------------------|-----------------|-----------|------------|------------|----------|----------|

A downside of using SQL to edit a table's columns is that we have to explicitly list out all of the columns again in the query.



In [61]:
# TODO: Create a function named station_ids_to_names_dict that takes in a station ID (numerical) and outputs its corresponding name (string).
def station_ids_to_names_dict(station_id):
  station_dict = dict(zip(stations_cleaned_df['station_id'], stations_cleaned_df['station_name']))
  return station_dict.get(station_id, "Unknown")

In [62]:
# DO NOT EDIT
try:
  duckdb.remove_function("station_ids_dict")
except:
  pass

# TODO: Use the station_ids_to_name_dict function to create a Duck DB function with the correct parameters.
duckdb.create_function("station_ids_dict", station_ids_to_names_dict, [int], str)

In [63]:
# TODO: Use the Duck DB function 'station_ids_dict' to create trips_final_df.
ids_to_names_query = """
SELECT
  trip_id,
  duration,
  station_ids_dict(CAST(start_station AS INT)) AS start_station,
  start_lat,
  start_lon,
  station_ids_dict(CAST(end_station AS INT)) AS end_station,
  end_lat,
  end_lon,
  bike_id,
  trip_route_category,
  passholder_type,
  bike_type,
  date_start,
  time_start,
  date_end,
  time_end,
FROM trips_cleaned_df
ORDER BY trip_id ASC
"""
trips_final_df = duckdb.sql(ids_to_names_query).df()

In [64]:
# 3 points
grader.grade(test_case_id = 'test_station_ids_to_names', answer = trips_final_df)

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 2.2.2 Creating the final query [6 points]

**TO-DO:**
- Using `trips_final_df`, find the top 5 most common end stations where Amtrak Station was the start_station where the duration is **at least 15 minutes**
  - Do **NOT** include Amtrak as an end station!
  - Note that duration is already measured in minutes, and that Amtrak's station_name is _"Amtrak 30th Street Station"_
- Name the final dataframe from the Duck DB query `end_station`.
- Ensure that `end_station` is first sorted by `Count` in descending order, and then sorted by `end_station` in ascending order. Conduct these processes in SQL (not Pandas).
- Please make sure to use `end_station_query` to save your SQL code as this is what will be graded.

**Final Schema (make sure you have the same column names and order!)**:
> end_station | Count
> --- | ---

In [65]:
# TODO: Get the same exact query, but only using SQL
end_station_query = """
SELECT end_station, COUNT(duration) as Count FROM trips_final_df

WHERE
  start_station = 'Amtrak 30th Street Station'
  AND duration >= 15
  AND end_station != 'Amtrak 30th Street Station'

GROUP BY end_station
ORDER BY Count DESC, end_station ASC
LIMIT 5
  """

end_station = duckdb.sql(end_station_query).df()

In [66]:
# 6 points
grader.grade(test_case_id = 'test_amtrak', answer = (end_station, end_station_query))

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.3 Relational Algebra [16 points]

#### 2.3.1 Inner-Join [4 points]
Inner-join: Using `stations_cleaned_df` and `trips_cleaned_df`, find all stations that had at least 100 trips leave from it.

**TODO**:
* Inner join `stations_cleaned_df` with `trips_cleaned_df` wherever the `station_id` in `stations_cleaned_df` is the same as the `start_station` in `trips_cleaned_df`
* Calculate the number of trips that departed from each station and store in column `trip_count`
* Select only the stations with `trip_count` greater than or equal to 100
* Sort by station name in ascending order

**Final Schema**:
>station_name | trip_count
>--- | ---

In [67]:
# TODO: Write SQL query to select all stations with at least 100 trips leaving from it
stations_query = """
SELECT station_name, COUNT(trip_id) AS trip_count
FROM trips_cleaned_df
INNER JOIN stations_cleaned_df
ON trips_cleaned_df.start_station = stations_cleaned_df.station_id
GROUP BY station_name
HAVING trip_count >= 100
ORDER BY station_name ASC

"""
stations_df = duckdb.sql(stations_query).df()

In [68]:
# 4 points

grader.grade(test_case_id = 'test_semi_join', answer = (stations_df, stations_query))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 2.3.2 Anti-Join [6 points]
Anti-join: Using `stations_cleaned_df` and `trips_cleaned_df`, find all bikes that have never arrived at Amtrak 30th Street Station.

**TODO**:
* Join `stations_cleaned_df` with `trips_cleaned_df` so every row in `stations_cleaned_df` has the name of the station it arrived at
* Create a df `amtrak_bikes` that contains one column with the `bike_id`s of each distinct bike that has arrived at Amtrak 30th Street Station
* Anti-join `trips_cleaned_df` with `amtrak_bikes` to get all bikes that have never arrived at Amtrak 30th Street Station
* Sort by bike_id in ascending order

**HINT:** In the second query, you can use the WHERE NOT EXISTS clause to select all rows in one table that don't have a corresponding row in another table. To see how to do this for an anti-join, read this article: https://vladmihalcea.com/sql-exists/


**Final Schema**:
>bike_id |
>--- |

In [69]:
# TODO: Write SQL query to create amtrak_bikes
amtrak_bikes_query = """
SELECT trips_cleaned_df.bike_id FROM trips_cleaned_df
INNER JOIN stations_cleaned_df
ON trips_cleaned_df.end_station = stations_cleaned_df.station_id
AND stations_cleaned_df.station_name = 'Amtrak 30th Street Station'
GROUP BY trips_cleaned_df.bike_id
ORDER BY trips_cleaned_df.bike_id ASC
"""
amtrak_bikes = duckdb.sql(amtrak_bikes_query).df()

In [70]:
# TODO: Write SQL query to select all bikes that have never arrived at Amtrak 30th Street Station
# by anti-joining trips_cleaned_df with amtrak_bikes
anti_amtrak_bikes_query = """
SELECT trips_cleaned_df.bike_id
FROM trips_cleaned_df
WHERE NOT EXISTS(
  SELECT bike_id FROM amtrak_bikes
  WHERE amtrak_bikes.bike_id = trips_cleaned_df.bike_id
)
GROUP BY trips_cleaned_df.bike_id
ORDER BY trips_cleaned_df.bike_id
"""
anti_amtrak_bikes = duckdb.sql(anti_amtrak_bikes_query).df()

In [71]:
# 6 points

grader.grade(test_case_id = 'test_anti_join', answer = (amtrak_bikes, anti_amtrak_bikes, amtrak_bikes_query, anti_amtrak_bikes_query))

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 2.3.3 Division [6 points]
Division: Using `stations_cleaned_df` and `trips_cleaned_df`, find all stations that have had all passholder types **arrive**.

**TODO**:
* Create a new DataFrame `stations_trips_df` by joining `stations_cleaned_df` with `trips_cleaned_df` that contains the `trip_id`, `station_name`, and `passholder_type` for every trip
* Select the names of stations that have had all passholder types **arrive**. Note: this is a simplified relational division question where table A = table B.
* Sort by station name in ascending order

**HINT: Try using HAVING() and COUNT() to avoid needing to take complicated cross-products for this question**

**Final Schema**:
>station_name |
>--- |

In [72]:
# TODO: Write a SQL query that selects the specified columns from the join of
# stations_cleaned_df and trips_cleaned_df

stations_trips_query = """
SELECT trips_cleaned_df.trip_id, stations_cleaned_df.station_name, trips_cleaned_df.passholder_type
FROM trips_cleaned_df, stations_cleaned_df
WHERE stations_cleaned_df.station_id = trips_cleaned_df.end_station
ORDER BY stations_cleaned_df.station_name
"""
stations_trips_df = duckdb.sql(stations_trips_query).df()

In [73]:
# TODO: Write a SQL query that finds the names of all stations that have had all
# passholder types arrive

all_bikes_query = """
SELECT station_name
FROM stations_trips_df
GROUP BY station_name
HAVING COUNT(DISTINCT passholder_type) = (SELECT COUNT(DISTINCT passholder_type) FROM trips_cleaned_df)
ORDER BY station_name ASC
"""
all_bikes_df = duckdb.sql(all_bikes_query).df()

In [74]:
# 4 points

grader.grade(test_case_id = 'test_division', answer = (stations_trips_df, all_bikes_df, stations_trips_query, all_bikes_query))

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.4 Weather and Trip Data [13 points]

#### 2.4.1 Trips and Weather Tables [3 points]

We will begin by slightly modifying `trips_final_df` and `actual_weather_cleaned_df`. We eventually want to merge using the `date` column from `weather_cleaned_df` and the `date_start` column from `trips_final_df`.

**TODO:** \\
* Create a new DataFrame `weather_merge_df`, containing columns `date`, `actual_max_temp` and `actual_precipitation` from `actual_weather_cleaned_df`
* Cast the `date` column in `weather_merge_df` to a string type so it can be used to join with the other DataFrame later

* Create a new DataFrame `trips_merge_df`, containing columns `date_start`,  `bike_type`, `trip_id`, `duration`

* Rename the `date_start` column from `trips_merge_df` to `date` and cast it to a string type so it can be used to join the two DataFrames later

In [93]:
# Make a copy so that it doesn't change the original df. Do not edit this line
weather_merge_df = actual_weather_cleaned_df.copy()

# TODO: Create weather_merge_df
weather_merge_df = weather_merge_df[["date", "actual_max_temp", "actual_precipitation"]]
weather_merge_df["date"] = weather_merge_df["date"].astype("string")
weather_merge_df

,date,actual_max_temp,actual_precipitation
0,2022-10-31,54,0.00
1,2022-10-30,60,0.00
2,2022-10-29,70,0.03
3,2022-10-28,76,0.00
4,2022-10-27,67,0.00
...,...,...,...
56,2022-09-05,89,0.00
57,2022-09-04,87,0.00
58,2022-09-03,85,0.00
59,2022-09-02,93,0.18


In [86]:
# Make a copy so that it doesn't change the original df. Do not edit this line
trips_merge_df = trips_final_df.copy()

# TODO: Create trips_merge_df
trips_merge_df  = trips_merge_df[["date_start", "bike_type", "trip_id", "duration"]]
trips_merge_df = trips_merge_df.rename(columns={"date_start": "date"})
trips_merge_df["date"] = trips_merge_df["date"].astype("string")

,date,bike_type,trip_id,duration
0,2020-10-01,standard,536546924,3
1,2020-10-01,standard,536546925,3
2,2020-10-01,standard,536546927,7
3,2020-10-01,standard,536546929,7
4,2020-10-01,electric,536546932,6
...,...,...,...,...
15250,2020-10-07,electric,540422839,100
15251,2020-10-07,electric,540422840,102
15252,2020-10-07,electric,540422841,102
15253,2020-10-07,standard,540422860,50


In [94]:
# 3 points
grader.grade(test_case_id = 'test_weather_trips_merge', answer = (weather_merge_df, trips_merge_df))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 2.4.2 Electric Riding [10 points]

Using `trips_merge_df` and `weather_merge_df`, *for each actual max temperature* where the **average actual_precipitation** for that actual max temperature is **less than** 0.3, find the percentage of rides that were of **electric** bike type on trips with a duration of greater than or equal to 20 minutes

* Name the actual max temperature column **Temperature**
* Name the percentages column column **Standard_Percentage**
* Make sure to multiply the percentage values by 100 so you get a float value of the percentage, not the decimal
* Sort by increasing temperature

**Final Schema (make sure you have the same column names and order!)**:
>Temperature | Standard_Percentage |
>--- | --- |

In [92]:
weather_merge_df

,date,actual_max_temp,actual_precipitation
0,2022-10-31,54,0.00
1,2022-10-30,60,0.00
2,2022-10-29,70,0.03
3,2022-10-28,76,0.00
4,2022-10-27,67,0.00
...,...,...,...
56,2022-09-05,89,0.00
57,2022-09-04,87,0.00
58,2022-09-03,85,0.00
59,2022-09-02,93,0.18


In [85]:
# TODO: Write a SQL query to select the data as described above
percentage_query = """

WITH AvgPrecipitation AS (
    SELECT
        w.actual_max_temp AS Temperature,
        AVG(w.actual_precipitation) AS Avg_Precipitation
    FROM weather_merge_df w
    GROUP BY w.actual_max_temp
    HAVING AVG(w.actual_precipitation) < 0.3
),
ElectricRidePercentages AS (
    SELECT
        w.actual_max_temp AS Temperature,
        100.0 * SUM(CASE WHEN t.bike_type = 'electric' THEN 1 ELSE 0 END) / COUNT(*) AS Standard_Percentage
    FROM trips_merge_df t
    JOIN weather_merge_df w ON t.date = w.date
    WHERE t.duration >= 20
    GROUP BY w.actual_max_temp
)

SELECT
    ElectricRidePercentages.Temperature,
    Standard_Percentage
FROM ElectricRidePercentages
JOIN AvgPrecipitation ON ElectricRidePercentages.Temperature = AvgPrecipitation.Temperature
ORDER BY ElectricRidePercentages.Temperature ASC;

"""

percentage_df_sql = duckdb.sql(percentage_query).df()
percentage_df_sql

,Temperature,Standard_Percentage


In [79]:
# 5 points
grader.grade(test_case_id = 'test_percentage_sql', answer = (percentage_df_sql, percentage_query))

Error: Test case failed. Test case function could not complete due to an error in your answer.
Error Hint: 0


Next, we want to compare writing a query like this in SQL to pandas. Perform the same operations on `trips_merge_df` and `weather_merge_df`, but this time using pandas. Save your results in a DataFrame called `percentage_df_pandas`, which should match the previous DataFrame exactly. Make sure to sort by increasing temperature and reset the index at the end.

In [89]:
# TODO: Create a DataFrame 'percentage_df_pandas' with the same result as the previous
# question, but using pandas instead of SQL

avg_precipitation = weather_merge_df.groupby("actual_max_temp")["actual_precipitation"].mean().reset_index()
avg_precipitation = avg_precipitation[avg_precipitation["actual_precipitation"] < 0.3]

merged_df = trips_merge_df.merge(weather_merge_df, left_on="date", right_on="date")

merged_df = merged_df.merge(avg_precipitation[["actual_max_temp"]], on="actual_max_temp")

percentage_df_pandas = (
    merged_df[merged_df["duration"] >= 20]
    .groupby("actual_max_temp")
    .agg(Standard_Percentage=("bike_type", lambda x: (x == "electric").mean() * 100))
    .reset_index()
)

percentage_df_pandas = percentage_df_pandas.rename(columns={"actual_max_temp": "Temperature"})

percentage_df_pandas = percentage_df_pandas.sort_values(by="Temperature", ascending=True).reset_index(drop=True)

In [90]:
# 5 points
grader.grade(test_case_id = 'test_percentage_pandas', answer = percentage_df_pandas)

Error: Test case failed. Test case function could not complete due to an error in your answer.
Error Hint: 0


## Part 3: Schema Design [25 points manually graded]

### 3.1 Functional Dependencies (5 points manually graded)
Let $A, B, C, D, E$ all be attributes in our schema.

Use Armstrong's axioms to prove that if $AB \rightarrow C$, $D \rightarrow E$ and $B$ is a subset of $E$, then $AD \rightarrow C$.

Review Lecture 6 (Relational schema) slide 50 if you need a refresher on Armstrong's axioms.

**Write your answer here:**

Step 1: $AB \rightarrow C$ (given)

Step 2: $D \rightarrow E$ (given)

Step 3: $B$ is a subset of $E$ (given)

Step 4: $E \rightarrow B$ (reflexivity of 3)

Step 5: $D \rightarrow B$ (transivity of 2 + 4)

Step 6: $AD \rightarrow AB$ (aug 5 with $A$)

Step 7: $AD \rightarrow C$ (transivity of 6 + 1)

### 3.2 3NF (10 points manually graded)
**Please answer the following question in the markdown cell below.**

Suppose $R = {A, B, C, D, E, F, G}$, and we have the following set of functional dependencies:

$A \rightarrow BC$

$B \rightarrow D$

$E \rightarrow AF$

$F \rightarrow G$

$G \rightarrow E$

Decompose $R$ into third normal form (3NF). Clearly mark the relations in your final answer, and specify their primary key.

**Write your answer here:**

It is apprent that the candidate keys are {E} and {F} and {G} when looking at R.

Since we can reach every other state from {E}, {E} is a candidate key. Using Armstrong's Axioms, we have $E \rightarrow AF,$ so $E \rightarrow A,$ and $E \rightarrow F,$ since $AF \rightarrow A$ and $AF \rightarrow F.$ We also have $A \rightarrow BC,$ so similarly $E \rightarrow B$ and $E \rightarrow C.$ In addition, $B \rightarrow D,$ so $E \rightarrow D.$ Finally, $E\rightarrow G$ since $F \rightarrow G.$

Since we can reach every other state from {F}, {F} is a candidate key. We have $F\rightarrow G$ and $G \rightarrow E,$ so we get $F \rightarrow E$ by transitivity. Since $F$ determines $E$ and $E$ determines all attributes of $R,$ we must have that $F$ determines $R$ as well.

Since $G\rightarrow E$, {G} is a candidate key. Since $G$ determines $E$ and $E$ determines all attributes of $R,$ we must have that $G$ determines $R$ as well.

Using the 3NF Decomposition Algorithm posted in Ed, we have FDs $A \rightarrow BC,$ $B \rightarrow D,$ $E \rightarrow AF,$ $F \rightarrow G,$ and $G \rightarrow E.$

Iterating, $R_1 = \{A,B,C\}$ since ABC is not in any other schema, and we can choose primary key $A.$

Similarly:

$R_2 = \{B,D\}$ since BD is not in any other schema, and we can choose primary key $B.$


$R_3 = \{E,A,F\}$ since EAF is not in any other schema, and we can choose primary key $E.$


$R_4 = \{F,G\}$ since FG is not in any other schema, and we can choose primary key $F.$


$R_5 = \{G,E\}$ since GE is not in any other schema, and we can choose primary key $G.$

Hence, we get decomposition:

$R_1 = \{A, B, C\} \text{ with primary key } A$

$R_2 = \{B, D\} \text{ with primary key } B$

$R_3 = \{E, A, F\} \text{ with primary key } E$

$R_4 = \{F, G\} \text{ with primary key } F$

$R_5 = \{G, E\} \text{ with primary key } G$

### 3.3 Entity Relationship Design (10 points manually graded)

**Please answer the following question in the markdown cell below by uploading an image of your solution. You can do this by:**


1. Upload images from your local machine by clicking on the files icon in the left sidebar
2. Click the **Upload** button
3. Select the image you want to upload after the file upload dialogue appears

Consider a university database that must be designed to manage students, classes, and professors. The system should be designed to capture the following requirements:

1. Every student has a unique student ID, name, major and
email address.
2. Every class has a unique class ID (e.g. CIS 5450), subject (e.g. CIS), number (e.g. 5450) and name (e.g. Big Data Analytics).
3. Each professor has a unique professor ID, name, and title.
4. Students can be in multiple classes.
5. Each class can have just one professor, but each professor can teach multiple classes.

Carefully consider the relationships that exist between students and classes, and classes and professors.

Design an ER diagram that captures the relationships above. Be sure to capture all entities, relationships, and cardinalities.

In [82]:
# Mount drive to give colab access to your other files

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [83]:
img = Image.open('ERDiagram.jpg')
img.show()
display(img)

FileNotFoundError: [Errno 2] No such file or directory: '/content/ERDiagram.jpg'

# HW Submission

Before you submit on Gradescope (you must submit your notebook to receive credit):


1.   Restart and Run-All to make sure there's nothing wrong with your notebook
2.   **Double check that you have the correct PennID (all numbers) in the autograder**.
3. Make sure you've run all the PennGrader cells
4. Go to the "File" tab at the top left, and download both the .ipynb and .py files, renaming them as "homework2.ipynb" and "homework2.py" respectively. Upload both files to Gradescope directly!
5. Check your .ipynb file is visible under the code tab. If there is a "large file error" go back and delete all occurances of printing an entire df, and/or make your figure sizes smaller, then resubmit. Then confirm the notebook preview is now visible in your Gradescope submission.

**You MUST check that your notebook displays ALL visualizations on the Gradescope preview AND verify that the autograder finishes running and gives you your expected score (not a 0).**

**Let the course staff know ASAP if you have any issues submitting, but otherwise best of luck! Congrats on finishing the HW.**